### Instructions for DAT processing
Niall Bourke ~ March 2019

The nuclear med physicists have a system called HERMES which they do the reconstructions of the the DAT scan before sending it to PACS. They are the holders of the DICOMS. Conveniently they are very nice and based on the first floor, block A in Hammersmith hospital.  

1. Collect reconstructbed DICOMS  
2. dcm2niix will convert to NIFTI so we can play in our sandbox  
3. fslreorient2std (may not be a necessary, but can help)  
4. Get MNI1mm striatal masks in native space to help with the manual registration of DAT to subjects T1   
5. Register DAT to native T1 using freeview 
6. Run DARTEL with all T1s and register to MNI space
7. Use the flowfields generated to register the DAT images to MNI space
8. Just because things need to be difficult the dimensions are off MNI space by 1mm..
9. Use flirt to resample within the same template.
10. Extract DAT values ( some methods included here )  
11. Do your flavour of stats   


SPM12 was run locally to translate everything into MNI space via DARTEL. 
This included resampling subCorticalNuclei into 1mm MNI space

### Freeview

Now there are striatal masks in each subjects native space we can open freeview with the subjects T1 and striatal masks & add the DAT scans to register

Viewing:

 - Leave T1 as greyscale  
 - Set mask as a distinct colour such as blue  
 - Set DAT as hot, red-yellow   

The scale for DAT will likely need to be changed in order to make it easier to visualise high uptake regions and remove noice. 

Below is a temporary alternative 

In [6]:
%%shell

# ----
# Set important variables 
# ----

# set path to data
workingDir=~/example
cd $workingDir
# set path to masks
templateDir=$workingDir/templates

# ----
# Register masks to subjects native space to aid manual registration of DAT
# ----

for sub in sub01;
    do
      flirt -in $templateDir/MNI152_T1_1mm.nii.gz -ref $workingDir/$sub/T1.nii.gz -omat $workingDir/$sub/MNI_${sub}.mat
      for mask in `ls $templateDir/masks`
      do
      echo $mask
      flirt -in $templateDir/masks/$mask -ref $workingDir/$sub/T1.nii.gz -applyxfm -init $workingDir/$sub/MNI_${sub}.mat -out $workingDir/$sub/native${mask}
      done
done

left_caudate_standard.nii.gz
left_putamen_standard.nii.gz
left_striatum_standard.nii.gz
right_caudate_standard.nii.gz
right_putamen_standard.nii.gz
right_striatum_standard.nii.gz


Prior to running the manual registration tool resample the DAT scan so it has the same dimensions as the T1. This will also bring it closer al

flirt -in DATscan -reg T1 -applyxfm -out resampled_DAT

## Translation using freeview (freesurfer tool)

- Load up the volumes in freeview  
-- T1: greyscale  
-- mask: Jet  
-- DAT: heat  

- Adjust the min, Max, offset for DAT to reduce the noise so only signal from striatal regions are visable. 

To translate:

click tools > transform volume > translate

Once we have saved the transformation matrix from the manaul adjustment of DAT to T1 then we can apply this using 

    mri_coreg --mov DAT.nii --ref T1.nii.gz --reg dat.lta

# SPM12 DARTEL pipeline

In [ ]:
%-----------------------------------------------------------------------
% Job saved on 11-Apr-2019 18:47:41 by cfg_util (rev $Rev: 6460 $)
% spm SPM - SPM12 (6906)
% cfg_basicio BasicIO - Unknown
%-----------------------------------------------------------------------

root = '/Users/nbourke/Desktop/CaseStudy/CONTROLS';
disp('Finding Files');

files = dir(root);
directoryNames = {files([files.isdir]).name};
directoryNames = directoryNames(~ismember(directoryNames,{'.','..'}));

for ii = 1:numel(directoryNames)
subj=(directoryNames(ii));
%T1 = strcat(root, filesep, subj, filesep, subj, '.nii');
flowField = strcat(root, filesep, subj, filesep, 'u_rc1', subj, '_Template.nii');

dirname = char(strcat(root, filesep, subj, filesep));
fname = dir([dirname,'DAT*'])
SBR=([fname.folder, filesep, fname.name])

spm('defaults','fmri');
spm_jobman('initcfg');

matlabbatch{1}.spm.tools.dartel.mni_norm.template = {'/Users/nbourke/Desktop/CaseStudy/Hypoxic/sub01/Template_6.nii'};


matlabbatch{1}.spm.tools.dartel.mni_norm.data.subj(1).flowfield = flowField;
matlabbatch{1}.spm.tools.dartel.mni_norm.data.subj(1).images = {SBR};


matlabbatch{1}.spm.tools.dartel.mni_norm.vox = [1 1 1];
matlabbatch{1}.spm.tools.dartel.mni_norm.bb = [NaN NaN NaN
                                               NaN NaN NaN];
matlabbatch{1}.spm.tools.dartel.mni_norm.preserve = 0;
matlabbatch{1}.spm.tools.dartel.mni_norm.fwhm = [8 8 8];

save second_batch matlabbatch;
load second_batch;
spm_jobman('run', matlabbatch);
clear matlabbatch 
end
disp('Finished Registration to MNI space');

### Resampling of dimnesions following DARTEL registration to MNI space
Everything sits in MNI space but for some reason the output of DARTEL is 1mm smaller than the MNI template (and subseqently masks) across all dimensions. To adress this flirt is used to resample to the same template. 

In [18]:
%%shell

# set path to data
workingDir=~/CaseStudy
cd $workingDir
template=$workingDir/templates/MNI152_T1_1mm.nii.gz 

for sub in `ls CONTROLS`;
    do
    #flirt -in $workingDir/CONTROLS/${sub}/sw${sub}.nii -ref $template -omat $workingDir/CONTROLS/${sub}/sw2MNI.mat
    #flirt -in $workingDir/CONTROLS/${sub}/sw${sub}.nii -ref $template -applyxfm -init $workingDir/CONTROLS/${sub}/sw2MNI.mat -out $workingDir/CONTROLS/${sub}/MNI1mm_${sub}_T1.nii        
    flirt -in $workingDir/CONTROLS/${sub}/swDAT*in_subj_T1_manual.nii  -ref $template -applyxfm -init $workingDir/CONTROLS/${sub}/sw2MNI.mat -out $workingDir/CONTROLS/${sub}/MNI1mm_${sub}_DAT.nii  
echo "if no warnings then compleated ${sub}.."
done

if no warnings then compleated T1_001_v1..
if no warnings then compleated T1_003_v1..
if no warnings then compleated T1_004_v1..
if no warnings then compleated T1_018_v1..
if no warnings then compleated T1_019_v1..
if no warnings then compleated T1_023_v1..
if no warnings then compleated T1_027_v1..
if no warnings then compleated T1_029_v1..
if no warnings then compleated T1_031_v1..
if no warnings then compleated T1_034_v1..
if no warnings then compleated T1_035_v1..
if no warnings then compleated T1_043_v1..
if no warnings then compleated T1_049_v1..
if no warnings then compleated T1_050_v1..
if no warnings then compleated T1_051_v1..
if no warnings then compleated T1_058_v1..
if no warnings then compleated T1_059_v1..
if no warnings then compleated T1_062_v1..
if no warnings then compleated T1_066_v1..
if no warnings then compleated T1_068_v1..


#### Get Signal Binding Ration for DAT

In [19]:
%%shell

# set path to data
workingDir=~/CaseStudy
template=$workingDir/templates/occ_mask_standard.nii.gz
cd $workingDir

for sub in `ls CONTROLS`;
    do
    # Calculate the count for the ROI
    function occ {
    fslstats $workingDir/CONTROLS/${sub}/MNI1mm_${sub}_DAT.nii -k ${template} -m
        }
    i=$(occ)
    
   # Create SBR image by subtracting and dividing the DAT input image from the values of the nonspecific tracer uptake region.
    fslmaths $workingDir/CONTROLS/${sub}/MNI1mm_${sub}_DAT.nii -sub ${i} -div ${i} $workingDir/CONTROLS/${sub}/MNI1mm_${sub}_SBR.nii 

done


#### Resample HBI patient

In [ ]:
%%shell

workingDir=~/CaseStudy
cd $workingDir
template=$workingDir/templates/MNI152_T1_1mm.nii.gz 

    #flirt -in $workingDir/Hypoxic/sub01/swT1.nii -ref $template -omat $workingDir/Hypoxic/sub01/sw2MNI.mat
    #flirt -in $workingDir/Hypoxic/sub01/swT1.nii -ref $template -applyxfm -init $workingDir/Hypoxic/sub01/sw2MNI.mat -out $workingDir/Hypoxic/sub01/MNI1mm_sub01_T1.nii        
    flirt -in $workingDir/Hypoxic/sub01/swSBR_manual_DAT2.nii  -ref $template -applyxfm -init $workingDir/Hypoxic/sub01/sw2MNI.mat -out $workingDir/Hypoxic/sub01/MNI1mm_sub01_SBR.nii  


### Resample subcortical Nuclei templates to same dimensions as MN1mm template

In [3]:
%%shell 

workingDir=~/CaseStudy
template=$workingDir/templates/MNI152_T1_1mm.nii.gz 


    flirt -in $workingDir/templates/subcorticalBrainNuclei/swCIT168toMNI152-2009c_T1w.nii -ref $template -omat $workingDir/templates/subcorticalBrainNuclei/sw2MNI.mat
    flirt -in $workingDir/templates/subcorticalBrainNuclei/swCIT168toMNI152-2009c_T1w.nii -ref $template -applyxfm -init $workingDir/templates/subcorticalBrainNuclei/sw2MNI.mat -out $workingDir/templates/subcorticalBrainNuclei/MNI1mm_T1   

cd $workingDir/templates/subcorticalBrainNuclei
for i in `ls swCIT168toMNI152_prob*`; 
    do
    flirt -in $workingDir/templates/subcorticalBrainNuclei/${i} -ref $template -applyxfm -init $workingDir/templates/subcorticalBrainNuclei/sw2MNI.mat -out $workingDir/templates/subcorticalBrainNuclei/MNI1mm_${i}  
done

In [10]:
%%shell 

workingDir=~/CaseStudy/templates/subcorticalBrainNuclei

cd $workingDir
for i in `ls MNI1mm_sw*`; 
    do
    fslmaths ${i} -thr 0.5 -bin ~/CaseStudy/templates/masks/${i}
    echo "${i} masked.."
done

MNI1mm_swCIT168toMNI152_prob_atlas_bilat_1mm_Ca.nii.gz masked..
MNI1mm_swCIT168toMNI152_prob_atlas_bilat_1mm_EXA.nii.gz masked..
MNI1mm_swCIT168toMNI152_prob_atlas_bilat_1mm_GPe.nii.gz masked..
MNI1mm_swCIT168toMNI152_prob_atlas_bilat_1mm_GPi.nii.gz masked..
MNI1mm_swCIT168toMNI152_prob_atlas_bilat_1mm_HN.nii.gz masked..
MNI1mm_swCIT168toMNI152_prob_atlas_bilat_1mm_HTH.nii.gz masked..
MNI1mm_swCIT168toMNI152_prob_atlas_bilat_1mm_MN.nii.gz masked..
MNI1mm_swCIT168toMNI152_prob_atlas_bilat_1mm_NAC.nii.gz masked..
MNI1mm_swCIT168toMNI152_prob_atlas_bilat_1mm_PBP.nii.gz masked..
MNI1mm_swCIT168toMNI152_prob_atlas_bilat_1mm_Pu.nii.gz masked..
MNI1mm_swCIT168toMNI152_prob_atlas_bilat_1mm_RN.nii.gz masked..
MNI1mm_swCIT168toMNI152_prob_atlas_bilat_1mm_SNc.nii.gz masked..
MNI1mm_swCIT168toMNI152_prob_atlas_bilat_1mm_SNr.nii.gz masked..
MNI1mm_swCIT168toMNI152_prob_atlas_bilat_1mm_STH.nii.gz masked..
MNI1mm_swCIT168toMNI152_prob_atlas_bilat_1mm_VeP.nii.gz masked..
MNI1mm_swCIT168toMNI152_prob_a

## Time to extract some values

In [21]:
%%shell

# set path to data
workingDir=~/CaseStudy
templateDir=$workingDir/templates/masks

cd $templateDir
for ROI in `ls`; 
do
    echo ${ROI::-7} > ${workingDir}/${ROI::-7}.txt
    for subj in `ls $workingDir/CONTROLS`;
    do
        fslstats ${workingDir}/CONTROLS/${subj}/MNI1mm_${sub}_SBR.nii -k $templateDir/${ROI} -M >> ${workingDir}/${ROI::-7}.txt
    done
done

paste -d "," `ls ${workingDir}/*.txt` > ${workingDir}/SBR_controls.csv

ERROR:: Empty mask image
ERROR:: Empty mask image
ERROR:: Empty mask image
ERROR:: Empty mask image
ERROR:: Empty mask image
ERROR:: Empty mask image
ERROR:: Empty mask image
ERROR:: Empty mask image
ERROR:: Empty mask image
ERROR:: Empty mask image
ERROR:: Empty mask image
ERROR:: Empty mask image
ERROR:: Empty mask image
ERROR:: Empty mask image
ERROR:: Empty mask image
ERROR:: Empty mask image
ERROR:: Empty mask image
ERROR:: Empty mask image
ERROR:: Empty mask image
ERROR:: Empty mask image
ERROR:: Empty mask image
ERROR:: Empty mask image
ERROR:: Empty mask image
ERROR:: Empty mask image
ERROR:: Empty mask image
ERROR:: Empty mask image
ERROR:: Empty mask image
ERROR:: Empty mask image
ERROR:: Empty mask image
ERROR:: Empty mask image
ERROR:: Empty mask image
ERROR:: Empty mask image
ERROR:: Empty mask image
ERROR:: Empty mask image
ERROR:: Empty mask image
ERROR:: Empty mask image
ERROR:: Empty mask image
ERROR:: Empty mask image
ERROR:: Empty mask image
ERROR:: Empty mask image


In [22]:
%%shell

# set path to data
workingDir=~/CaseStudy
templateDir=$workingDir/templates/masks

cd $templateDir
for ROI in `ls`; 
do
    echo ${ROI::-7} > ${workingDir}/HBI_${ROI::-7}.txt
    for subj in sub01;
    do
        fslstats ${workingDir}/Hypoxic/${subj}/MNI1mm_${subj}_SBR.nii.gz -k $templateDir/${ROI} -M >> ${workingDir}/HBI_${ROI::-7}.txt
    done
done

paste -d "," `ls ${workingDir}/HBI*.txt` > ${workingDir}/SBR_hypoxic.csv

ERROR:: Empty mask image
ERROR:: Empty mask image
ERROR:: Empty mask image
ERROR:: Empty mask image
ERROR:: Empty mask image
ERROR:: Empty mask image
ERROR:: Empty mask image
ERROR:: Empty mask image
ERROR:: Empty mask image
ERROR:: Empty mask image
ERROR:: Empty mask image
ERROR:: Empty mask image
ERROR:: Empty mask image
ERROR:: Empty mask image
